In [1]:
import os
import re

In [2]:
os.getcwd()
path = '/Volumes/JK/Swizzl/swizzl/'
os.chdir(path)

In [4]:
from bs4 import BeautifulSoup
import requests 
from services.mlearning import sentiment as st
from services.mlearning import prof

In [5]:
def linkText(link):
    url = "{}".format(link)
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"lxml")
    article_text = ''
    mylink = soup.find('div').findAll('p')
    for i in mylink:
        article_text += ''.join(i.findAll(text = True))
    return article_text

In [6]:
def YahooFetch():
    url = "https://news.yahoo.com/rss/"
    url = requests.get(url)
    soup = BeautifulSoup(url.text,'xml')
    
    try:
        # Fetch All Required items
        titles = soup.findAll('title')
        links = soup.findAll('link')
        pubDates = soup.findAll('pubDate')

    except Exception as e :
        
        # Return Empty if titles, links, descriptions, pubdates not found
        print(e)
        return "Error"

    # We don't want the first elements as these are just metadata
    
    titles.pop(0)
    titles.pop(0)
    links.pop(0)
    links.pop(0)
    pubDates.pop(0)

    #Dictionary to hold crawled information
    


    FeedDict = {}
    temp = []
    
    
    try:
        # Add titles to the Dictionary
        
        for i in titles:
            temp.append(i.get_text())
            FeedDict['title'] = temp
            
        # Add link and details regarding text contetn @ link to the Dictionary
        
        temp = []
        temptext = []
        tbscore = []
        vadscore = []
        profvalue = []
        for i in links:
            
            # Append Links
            temp.append(i.get_text())
            
            # Append Text Content from Links
            textval = linkText(i.get_text())
            textval = re.sub('\"','\\"',textval)
            textval = " \" " + textval + " \" "
            temptext.append(textval)
            
            # Find Subjectivity, Objectivity of text content
            score = st.sentimentTB(textval)
            tbscore.append(score)
            
            # Find Sentiment of text content
            score = st.sentimentVader(textval)
            vadscore.append(score)
            
            # Find Profanity Score of content
            textval = [textval]
            score = float(prof.predProf(textval))
            profvalue.append(score)
            
            # Add to Feeds Dictionary 
            FeedDict['link'] = temp
            FeedDict['linktext'] = temptext
            FeedDict['tbScore'] = tbscore
            FeedDict['vaderScore'] = vadscore
            FeedDict['prof'] = profvalue
            
        # Add Published Dates to the Dictionary
        temp = []
        for i in pubDates:
            temp.append(i.get_text())
            FeedDict['pubdate'] = temp
        
        print("Success")
    except Exception as e :
        print(e)
        return "Error"
    
    return FeedDict

In [ ]:
import time
start_time = time.time()
FeedDict = YahooFetch()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for i in range(0,54):
    print(FeedDict['title'][i])

In [ ]:
count = 0
for i in range(0,len(FeedDict['title'])):
    if(re.findall("^https://news.yahoo.com",FeedDict['link'][i])):
        count = count + 1
print(count)

In [ ]:
text =  " Police breached a Hong Kong university campus held by protesters early Monday after an all-night siege that included firing repeated barrages of tear gas and water cannons.Anti-government protesters have barricaded themselves inside Hong Kong Polytechnic University for days. Police surrounded the area Sunday night and began moving in after issuing an ultimatum for people to leave the area. The crowd wore raincoats and carried umbrellas to shield themselves.Riot officers broke in one entrance before dawn as fires raged inside and outside the school, but they didn’t appear to get very far. Fiery explosions could be seen as protesters responded with gasoline bombs. Police, who have warned that everyone in the area could be charged with rioting, reportedly made a handful of arrests.At daybreak, protesters remained in control of most of the campus. In one outdoor area, some demonstrators made gasoline bombs while others dozed while wearing gas masks. Two walked about with bows and quivers of arrows, while many stared at their smartphones.A lull settled on the area as the president of the university said in a video message that that police have agreed to suspend their use of force.Jin-Guang Teng said police would allow protesters to leave and he would accompany them to the police station to ensure their cases would be processed fairly.“I hope that you will accept the proposed temporary suspension of force and leave the campus in a peaceful manner,” he said.It seemed unlikely the protesters would accept the offer given that they would all likely be arrested.A few hundred streamed out of the campus about 8:15 a.m. in an apparent bid to escape, but they were driven back by police tear gas. Some wearing gas masks calmly picked up smoking tear gas canisters and dropped them into heavy-duty bags, but the protesters decided to retreat with a phalanx of officers lined up across the road in the distance.On Sunday, protesters used bows and arrows, and one arrow struck a media liaison officer in the calf. Photos on the department’s Facebook page show the arrow sticking out of the back of the officer’s leg through his pants.As riot police moved in from all sides, some protesters retreated inside the university. Others set fires on bridges leading to it.A huge blaze burned along much of a long footbridge that connects a train station to the campus over the approach to the Cross-Harbour Tunnel, a major road under Hong Kong’s harbor that has been blocked by protesters for days.The use of bows and arrows and gasoline bombs was a sharp escalation of violence by the protesters, who are trying to keep the pressure on Hong Kong leaders, who have rejected most of their demands.The protests started peacefully in early June, sparked by proposed legislation that would have criminal suspects to be extradited to the mainland. But by the time the bill was withdrawn, the protests had hardened and broadened into a resistance movement against the territory’s government and Beijing.Activists see the extradition bill as an example of Hong Kong’s eroding autonomy under Beijing rule since the 1997 handover from colonial power Britain. (AP)See more news-related photo galleries and follow us on Yahoo News Photo Twitter and Tumblr._____Download the Yahoo News app to customize your experience.See more galleries from Yahoo News Photo:PHOTOS: Deadly shooting at California football partyPHOTOS: 10 bronze statues of inspirational women in NYC by Statues for Equality\"PHOTOS: California high school shootingPHOTOS: Layered portraits of Hong Kong's masked protestersPHOTOS: Venice flooded from rising tides and rainPHOTOS: Veterans Day 2019 observed at parades, memorials across the U.S.PHOTOS: For Syrian Kurds, and aid workers – the ‘safe zone’ is not so safe " 

In [ ]:
text = [text]

In [ ]:
float(prof.predProf(text))

In [ ]:
print(FeedDict['linktext'][0])

In [ ]:
# print(FeedDict['title'][0])
# print(FeedDict['link'][0])
# print(FeedDict['linktext'][0])
# print(FeedDict['pubdate'][0])
print(FeedDict['tbScore'][0])
print(FeedDict['vaderScore'][0])
print(FeedDict['prof'][0])

In [ ]:
import re
docs = []
for i in range(0,len(FeedDict['title'])):
    text = re.sub('&#39','',FeedDict['title'][i])
    docs.append(text)

In [ ]:
docs

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in docs] 

In [ ]:
doc_clean

In [ ]:
import gensim
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel

In [ ]:
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

In [ ]:
topics = ldamodel.print_topics(num_topics=3)
for topic in topics:
    print(topic)

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer 
lmtzr=WordNetLemmatizer()

In [ ]:
from time import time

In [ ]:
start_time = time.time()
print(lmtzr.lemmatize('cats'))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
print(lmtzr.lemmatize('dogs'))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
print(lmtzr.lemmatize('impeachment'))
print("--- %s seconds ---" % (time.time() - start_time))